# import necessary packages

In [1]:
from tensorflow.keras import layers
import tensorflow.keras as keras
import tensorflow as tf

import tensorflow_datasets as tfds

from keras.preprocessing.sequence import pad_sequences

import pandas as pd
import numpy as np

Using TensorFlow backend.


# imdb_reviews/submwords8k - 8000 vocab size
# read train_data, test_data and info

In [2]:
(train_data, test_data), info = tfds.load('imdb_reviews/subwords8k', split=(tfds.Split.TRAIN, tfds.Split.TEST), 
                                          with_info=True, as_supervised=True)

# Type of 'info'

In [3]:
type(info)

tensorflow_datasets.core.dataset_info.DatasetInfo

# What is in 'info'

In [4]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(None,), dtype=tf.int64, encoder=<SubwordTextEncoder vocab_size=8185>),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Pot

# info.features

In [5]:
info.features

FeaturesDict({
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(None,), dtype=tf.int64, encoder=<SubwordTextEncoder vocab_size=8185>),
})

# info.features['text']

In [6]:
info.features['text']

Text(shape=(None,), dtype=tf.int64, encoder=<SubwordTextEncoder vocab_size=8185>)

# info.features['text'].encoder

In [7]:
info.features['text'].encoder

<SubwordTextEncoder vocab_size=8185>

In [8]:
encoder = info.features['text'].encoder

# info.features['text'].encoder.vocab_size

In [9]:
encoder.vocab_size

8185

# info.features['text'].encoder.subwords - after tokenization

In [10]:
info.features['text'].encoder.subwords

['the_',
 ', ',
 '. ',
 'a_',
 'and_',
 'of_',
 'to_',
 's_',
 'is_',
 'br',
 'in_',
 'I_',
 'that_',
 'this_',
 'it_',
 ' /><',
 ' />',
 'was_',
 'The_',
 'as_',
 't_',
 'with_',
 'for_',
 '.<',
 'on_',
 'but_',
 'movie_',
 ' (',
 'are_',
 'his_',
 'have_',
 'film_',
 'not_',
 'ing_',
 'be_',
 'ed_',
 'you_',
 ' "',
 'it',
 'd_',
 'an_',
 'he_',
 'by_',
 'at_',
 'one_',
 'who_',
 'y_',
 'from_',
 'e_',
 'or_',
 'all_',
 'like_',
 'they_',
 '" ',
 'so_',
 'just_',
 'has_',
 ') ',
 'her_',
 'about_',
 'out_',
 'This_',
 'some_',
 'ly_',
 'movie',
 'film',
 'very_',
 'more_',
 'It_',
 'would_',
 'what_',
 'when_',
 'which_',
 'good_',
 'if_',
 'up_',
 'only_',
 'even_',
 'their_',
 'had_',
 'really_',
 'my_',
 'can_',
 'no_',
 'were_',
 'see_',
 'she_',
 '? ',
 'than_',
 '! ',
 'there_',
 'get_',
 'been_',
 'into_',
 ' - ',
 'will_',
 'much_',
 'story_',
 'because_',
 'ing',
 'time_',
 'n_',
 'we_',
 'ed',
 'me_',
 ': ',
 'most_',
 'other_',
 'don',
 'do_',
 'm_',
 'es_',
 'how_',
 'also

# type of 'train_data'

In [11]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

# see what is in train_data

In [12]:
for x,y in train_data:
    
    print(x)
    print(y)
    
    break

tf.Tensor(
[  62    9    4  301 4161  267  148    1 3240 1779  787    3   62 2315
  260 7968   21 1240   20  445   20  261  204    2    5   15  635 7742
  149   97  101   25  184 4127    3 4666 7913  690   25    9  176    1
  175  233   60   14  694    2   26   30 1858 3162   34    9 3636   40
  267   11   14 2362 8050    3   19  695   29   51 1816 7138    2   26
   14  101    1  397 3953  199  615    3   19  328    9 3362 4712 7961
    5    1  184    9   77 4167   64 1152    2   55   12  459 1461    6
 1713 3326   11 1147    7 1464 5691 7961  421 8026   38 1746 1074  618
   54   65    3 1987    2    1 3326   29  214    5  318 2338 3960 8039
    2    5  325    2   14   32    9 4157 7961   44  883 7975], shape=(138,), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)


# types contained in train_data 

In [13]:
for x,y in train_data:
    
    print(type(x))
    print(type(y))
    
    break

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


# this is how to take the array in an EagerTensor element in train_data

In [14]:
for x,y in train_data:
    
    print(x.numpy())
    
    break

[  62    9    4  301 4161  267  148    1 3240 1779  787    3   62 2315
  260 7968   21 1240   20  445   20  261  204    2    5   15  635 7742
  149   97  101   25  184 4127    3 4666 7913  690   25    9  176    1
  175  233   60   14  694    2   26   30 1858 3162   34    9 3636   40
  267   11   14 2362 8050    3   19  695   29   51 1816 7138    2   26
   14  101    1  397 3953  199  615    3   19  328    9 3362 4712 7961
    5    1  184    9   77 4167   64 1152    2   55   12  459 1461    6
 1713 3326   11 1147    7 1464 5691 7961  421 8026   38 1746 1074  618
   54   65    3 1987    2    1 3326   29  214    5  318 2338 3960 8039
    2    5  325    2   14   32    9 4157 7961   44  883 7975]


# take the text sequences (integer encoded sequences) in train_data into a list
# also take the labels into a list

In [15]:
train_text_sequences = list()
train_labels = list()
for x,y in train_data:
    train_text_sequences.append(x.numpy())
    train_labels.append(y.numpy())

# take the text sequences list and labels list into a dataframe

In [16]:
dct_train_labeled_text = {'text sequences': train_text_sequences, 'labels': train_labels}

In [17]:
df_train_labeled_text = pd.DataFrame(dct_train_labeled_text)

In [18]:
df_train_labeled_text.head()

,text sequences,labels
0,"[62, 9, 4, 301, 4161, 267, 148, 1, 3240, 1779,...",0
1,"[2130, 99, 12, 18, 55, 2554, 2, 3508, 5, 7995,...",0
2,"[4491, 40, 6, 1, 7450, 34, 4798, 80, 4, 238, 7...",1
3,"[398, 105, 14, 9, 4, 98, 13, 732, 22, 63, 333,...",0
4,"[62, 9, 33, 4, 132, 65, 3, 69, 2494, 1, 293, 5...",1


# the text sequences are of variable length

In [19]:
length_of_train_text = list()
for i,r in df_train_labeled_text.iterrows():
    length_of_train_text.append(len(r['text sequences']))

In [20]:
length_of_train_text[:5]

[138, 200, 708, 146, 126]

In [21]:
max(length_of_train_text)

3944

In [22]:
min(length_of_train_text)

11

# take the text sequences and labels in the test_data into lists

In [23]:
test_text_sequences = list()
test_labels = list()
for x,y in test_data:
    test_text_sequences.append(x.numpy())
    test_labels.append(y.numpy())

# convert these lists into a dataframe 

In [24]:
dct_test_labeled_text = {'text sequences': test_text_sequences, 'labels': test_labels}

In [25]:
df_test_labeled_text = pd.DataFrame(dct_test_labeled_text)

In [26]:
df_test_labeled_text.head()

,text sequences,labels
0,"[69, 5680, 22, 155, 6819, 7961, 6197, 309, 215...",1
1,"[12, 582, 448, 14, 44, 82, 1080, 2667, 464, 44...",0
2,"[62, 631, 32, 620, 3783, 8, 84, 3877, 190, 3, ...",1
3,"[6270, 762, 21, 5290, 6724, 3077, 8, 11, 59, 4...",0
4,"[8002, 7968, 111, 81, 33, 215, 7, 613, 82, 101...",0


# length of text sequences in test_data are of variable length

In [27]:
length_of_test_text = list()
for i,r in df_test_labeled_text.iterrows():
    length_of_test_text.append(len(r['text sequences']))

In [28]:
length_of_test_text[:5]

[226, 248, 203, 163, 159]

In [29]:
max(length_of_test_text)

3454

In [30]:
min(length_of_test_text)

9

# take the train text sequences and test text sequences as lists

In [31]:
X_train = list(df_train_labeled_text['text sequences'])

In [32]:
X_test = list(df_test_labeled_text['text sequences'])

# pad the text sequences
# add 0s to text sequences having length less than the max length
# truncate the text sequences having length more than the max length to max length

In [33]:
maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# after padding see the lengths of text sequences in train and test data 

In [34]:
train_post_padding_length = list()
for a in X_train:
    train_post_padding_length.append(len(a))
    
test_post_padding_length = list()
for a in X_test:
    test_post_padding_length.append(len(a))

In [35]:
train_post_padding_length[:5]

[100, 100, 100, 100, 100]

In [36]:
test_post_padding_length[:5]

[100, 100, 100, 100, 100]

# build the neural network model_1 - global average pooling 1D

In [37]:
embedding_dim=16

model_1 = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1)
])

model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 131,249
Trainable params: 131,249
Non-trainable params: 0
_________________________________________________________________


# take the train labels as a numpy array

In [38]:
y_train = list(df_train_labeled_text['labels'])
y_train = np.array(y_train)

# take the test labels as a numpy array 

In [39]:
y_test = list(df_test_labeled_text['labels'])
y_test = np.array(y_test)

# compile the model_1

In [40]:
model_1.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# train the model_1

In [41]:
history_1 = model_1.fit(X_train, y_train, epochs=5)

Train on 25000 samples
Epoch 1/5
25000/25000 [==============================] - ETA: 5:49 - loss: 0.6932 - accuracy: 0.40 - ETA: 12s - loss: 0.6930 - accuracy: 0.4917 - ETA: 6s - loss: 0.6925 - accuracy: 0.504 - ETA: 4s - loss: 0.6918 - accuracy: 0.50 - ETA: 3s - loss: 0.6905 - accuracy: 0.50 - ETA: 3s - loss: 0.6889 - accuracy: 0.50 - ETA: 2s - loss: 0.6866 - accuracy: 0.49 - ETA: 2s - loss: 0.6836 - accuracy: 0.49 - ETA: 2s - loss: 0.6797 - accuracy: 0.49 - ETA: 1s - loss: 0.6749 - accuracy: 0.49 - ETA: 1s - loss: 0.6685 - accuracy: 0.49 - ETA: 1s - loss: 0.6605 - accuracy: 0.50 - ETA: 1s - loss: 0.6529 - accuracy: 0.51 - ETA: 1s - loss: 0.6451 - accuracy: 0.52 - ETA: 1s - loss: 0.6377 - accuracy: 0.53 - ETA: 0s - loss: 0.6277 - accuracy: 0.55 - ETA: 0s - loss: 0.6195 - accuracy: 0.56 - ETA: 0s - loss: 0.6113 - accuracy: 0.57 - ETA: 0s - loss: 0.6038 - accuracy: 0.58 - ETA: 0s - loss: 0.5959 - accuracy: 0.59 - ETA: 0s - loss: 0.5880 - accuracy: 0.60 - ETA: 0s - loss: 0.5808 - accurac

# evaluate the trained model_1 on test data

In [42]:
model_1.evaluate(X_test, y_test)

25000/25000 [==============================] - ETA: 46s - loss: 0.3395 - accuracy: 0.843 - ETA: 1s - loss: 0.4915 - accuracy: 0.802 - ETA: 0s - loss: 0.4938 - accuracy: 0.79 - ETA: 0s - loss: 0.4890 - accuracy: 0.79 - ETA: 0s - loss: 0.4847 - accuracy: 0.80 - ETA: 0s - loss: 0.4847 - accuracy: 0.80 - ETA: 0s - loss: 0.4834 - accuracy: 0.80 - ETA: 0s - loss: 0.4748 - accuracy: 0.80 - ETA: 0s - loss: 0.4724 - accuracy: 0.80 - ETA: 0s - loss: 0.4718 - accuracy: 0.80 - ETA: 0s - loss: 0.4735 - accuracy: 0.80 - 1s 25us/sample - loss: 0.4731 - accuracy: 0.8057


[0.47305155502319335, 0.80572]

# build the neural network model_2 - lstm

In [43]:
embedding_dim=16

model_2 = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.LSTM(128),
  layers.Dense(16, activation='relu'),
  layers.Dense(1)
])

model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               74240     
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 207,281
Trainable params: 207,281
Non-trainable params: 0
_________________________________________________________________


# compile the model_2

In [44]:
model_2.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# train the model_2

In [45]:
history_2 = model_2.fit(X_train, y_train, epochs=5)

Train on 25000 samples
Epoch 1/5


 5920/25000 [======>.......................] - ETA: 18:48 - loss: 0.6927 - accuracy: 0.625 - ETA: 9:49 - loss: 0.6921 - accuracy: 0.593 - ETA: 6:48 - loss: 0.6923 - accuracy: 0.57 - ETA: 5:17 - loss: 0.6912 - accuracy: 0.58 - ETA: 4:22 - loss: 0.6923 - accuracy: 0.56 - ETA: 3:46 - loss: 0.6915 - accuracy: 0.56 - ETA: 3:19 - loss: 0.6925 - accuracy: 0.55 - ETA: 3:00 - loss: 0.6923 - accuracy: 0.55 - ETA: 2:45 - loss: 0.6936 - accuracy: 0.53 - ETA: 2:33 - loss: 0.6948 - accuracy: 0.51 - ETA: 2:23 - loss: 0.6955 - accuracy: 0.50 - ETA: 2:14 - loss: 0.6957 - accuracy: 0.50 - ETA: 2:07 - loss: 0.6958 - accuracy: 0.49 - ETA: 2:01 - loss: 0.6955 - accuracy: 0.50 - ETA: 1:56 - loss: 0.6952 - accuracy: 0.50 - ETA: 1:51 - loss: 0.6949 - accuracy: 0.50 - ETA: 1:47 - loss: 0.6947 - accuracy: 0.51 - ETA: 1:44 - loss: 0.6945 - accuracy: 0.51 - ETA: 1:40 - loss: 0.6943 - accuracy: 0.51 - ETA: 1:37 - loss: 0.6942 - accuracy: 0.51 - ETA: 1:35 - loss: 0.6943 - accuracy: 0.51 - ETA: 1:32 - loss: 0.6943 -

11872/25000 [=============>................] - ETA: 39s - loss: 0.6725 - accuracy: 0.534 - ETA: 39s - loss: 0.6720 - accuracy: 0.535 - ETA: 39s - loss: 0.6717 - accuracy: 0.536 - ETA: 39s - loss: 0.6708 - accuracy: 0.536 - ETA: 39s - loss: 0.6710 - accuracy: 0.537 - ETA: 39s - loss: 0.6710 - accuracy: 0.537 - ETA: 39s - loss: 0.6706 - accuracy: 0.538 - ETA: 39s - loss: 0.6702 - accuracy: 0.539 - ETA: 38s - loss: 0.6697 - accuracy: 0.539 - ETA: 38s - loss: 0.6686 - accuracy: 0.541 - ETA: 38s - loss: 0.6677 - accuracy: 0.542 - ETA: 38s - loss: 0.6672 - accuracy: 0.543 - ETA: 38s - loss: 0.6666 - accuracy: 0.543 - ETA: 38s - loss: 0.6650 - accuracy: 0.545 - ETA: 38s - loss: 0.6647 - accuracy: 0.546 - ETA: 38s - loss: 0.6633 - accuracy: 0.547 - ETA: 38s - loss: 0.6627 - accuracy: 0.548 - ETA: 38s - loss: 0.6618 - accuracy: 0.550 - ETA: 38s - loss: 0.6608 - accuracy: 0.551 - ETA: 38s - loss: 0.6601 - accuracy: 0.552 - ETA: 38s - loss: 0.6602 - accuracy: 0.553 - ETA: 37s - loss: 0.6594 - acc

17824/25000 [====================>.........] - ETA: 26s - loss: 0.5870 - accuracy: 0.644 - ETA: 26s - loss: 0.5866 - accuracy: 0.644 - ETA: 26s - loss: 0.5859 - accuracy: 0.645 - ETA: 26s - loss: 0.5856 - accuracy: 0.645 - ETA: 26s - loss: 0.5855 - accuracy: 0.645 - ETA: 26s - loss: 0.5851 - accuracy: 0.646 - ETA: 26s - loss: 0.5848 - accuracy: 0.646 - ETA: 26s - loss: 0.5844 - accuracy: 0.647 - ETA: 26s - loss: 0.5841 - accuracy: 0.647 - ETA: 26s - loss: 0.5833 - accuracy: 0.648 - ETA: 26s - loss: 0.5828 - accuracy: 0.648 - ETA: 26s - loss: 0.5826 - accuracy: 0.649 - ETA: 26s - loss: 0.5826 - accuracy: 0.649 - ETA: 26s - loss: 0.5822 - accuracy: 0.649 - ETA: 26s - loss: 0.5817 - accuracy: 0.650 - ETA: 26s - loss: 0.5816 - accuracy: 0.650 - ETA: 25s - loss: 0.5817 - accuracy: 0.650 - ETA: 25s - loss: 0.5812 - accuracy: 0.650 - ETA: 25s - loss: 0.5809 - accuracy: 0.651 - ETA: 25s - loss: 0.5802 - accuracy: 0.651 - ETA: 25s - loss: 0.5796 - accuracy: 0.652 - ETA: 25s - loss: 0.5790 - acc

23936/25000 [===========================>..] - ETA: 14s - loss: 0.5473 - accuracy: 0.689 - ETA: 14s - loss: 0.5472 - accuracy: 0.689 - ETA: 14s - loss: 0.5474 - accuracy: 0.689 - ETA: 14s - loss: 0.5472 - accuracy: 0.689 - ETA: 14s - loss: 0.5471 - accuracy: 0.689 - ETA: 14s - loss: 0.5467 - accuracy: 0.689 - ETA: 14s - loss: 0.5463 - accuracy: 0.690 - ETA: 14s - loss: 0.5460 - accuracy: 0.690 - ETA: 14s - loss: 0.5459 - accuracy: 0.690 - ETA: 14s - loss: 0.5459 - accuracy: 0.691 - ETA: 14s - loss: 0.5455 - accuracy: 0.691 - ETA: 14s - loss: 0.5454 - accuracy: 0.691 - ETA: 14s - loss: 0.5452 - accuracy: 0.691 - ETA: 14s - loss: 0.5450 - accuracy: 0.691 - ETA: 13s - loss: 0.5449 - accuracy: 0.692 - ETA: 13s - loss: 0.5448 - accuracy: 0.692 - ETA: 13s - loss: 0.5448 - accuracy: 0.692 - ETA: 13s - loss: 0.5444 - accuracy: 0.692 - ETA: 13s - loss: 0.5443 - accuracy: 0.692 - ETA: 13s - loss: 0.5439 - accuracy: 0.693 - ETA: 13s - loss: 0.5435 - accuracy: 0.693 - ETA: 13s - loss: 0.5433 - acc

...

 5952/25000 [======>.......................] - ETA: 52s - loss: 0.2896 - accuracy: 0.906 - ETA: 55s - loss: 0.2009 - accuracy: 0.921 - ETA: 56s - loss: 0.1688 - accuracy: 0.937 - ETA: 56s - loss: 0.1480 - accuracy: 0.953 - ETA: 57s - loss: 0.1396 - accuracy: 0.956 - ETA: 57s - loss: 0.1604 - accuracy: 0.947 - ETA: 57s - loss: 0.1580 - accuracy: 0.955 - ETA: 57s - loss: 0.1509 - accuracy: 0.957 - ETA: 57s - loss: 0.1475 - accuracy: 0.958 - ETA: 57s - loss: 0.1489 - accuracy: 0.956 - ETA: 57s - loss: 0.1471 - accuracy: 0.957 - ETA: 57s - loss: 0.1665 - accuracy: 0.945 - ETA: 57s - loss: 0.1785 - accuracy: 0.939 - ETA: 57s - loss: 0.1879 - accuracy: 0.937 - ETA: 57s - loss: 0.1891 - accuracy: 0.939 - ETA: 57s - loss: 0.1840 - accuracy: 0.943 - ETA: 57s - loss: 0.1849 - accuracy: 0.943 - ETA: 57s - loss: 0.1798 - accuracy: 0.944 - ETA: 57s - loss: 0.1830 - accuracy: 0.944 - ETA: 57s - loss: 0.1900 - accuracy: 0.942 - ETA: 57s - loss: 0.1938 - accuracy: 0.942 - ETA: 57s - loss: 0.1941 - acc

11904/25000 [=============>................] - ETA: 46s - loss: 0.1658 - accuracy: 0.941 - ETA: 46s - loss: 0.1656 - accuracy: 0.942 - ETA: 46s - loss: 0.1658 - accuracy: 0.942 - ETA: 46s - loss: 0.1656 - accuracy: 0.942 - ETA: 46s - loss: 0.1653 - accuracy: 0.942 - ETA: 46s - loss: 0.1654 - accuracy: 0.942 - ETA: 46s - loss: 0.1648 - accuracy: 0.942 - ETA: 46s - loss: 0.1647 - accuracy: 0.942 - ETA: 46s - loss: 0.1654 - accuracy: 0.942 - ETA: 46s - loss: 0.1653 - accuracy: 0.942 - ETA: 45s - loss: 0.1650 - accuracy: 0.942 - ETA: 45s - loss: 0.1647 - accuracy: 0.942 - ETA: 45s - loss: 0.1644 - accuracy: 0.942 - ETA: 45s - loss: 0.1639 - accuracy: 0.942 - ETA: 45s - loss: 0.1644 - accuracy: 0.942 - ETA: 45s - loss: 0.1637 - accuracy: 0.942 - ETA: 45s - loss: 0.1637 - accuracy: 0.942 - ETA: 45s - loss: 0.1643 - accuracy: 0.942 - ETA: 45s - loss: 0.1638 - accuracy: 0.942 - ETA: 45s - loss: 0.1643 - accuracy: 0.942 - ETA: 45s - loss: 0.1639 - accuracy: 0.942 - ETA: 45s - loss: 0.1634 - acc

17856/25000 [====================>.........] - ETA: 32s - loss: 0.1751 - accuracy: 0.935 - ETA: 32s - loss: 0.1754 - accuracy: 0.935 - ETA: 32s - loss: 0.1754 - accuracy: 0.935 - ETA: 32s - loss: 0.1756 - accuracy: 0.935 - ETA: 32s - loss: 0.1758 - accuracy: 0.935 - ETA: 32s - loss: 0.1760 - accuracy: 0.935 - ETA: 32s - loss: 0.1763 - accuracy: 0.934 - ETA: 32s - loss: 0.1762 - accuracy: 0.935 - ETA: 31s - loss: 0.1761 - accuracy: 0.935 - ETA: 31s - loss: 0.1760 - accuracy: 0.935 - ETA: 31s - loss: 0.1765 - accuracy: 0.934 - ETA: 31s - loss: 0.1767 - accuracy: 0.934 - ETA: 31s - loss: 0.1765 - accuracy: 0.934 - ETA: 31s - loss: 0.1764 - accuracy: 0.934 - ETA: 31s - loss: 0.1763 - accuracy: 0.934 - ETA: 31s - loss: 0.1762 - accuracy: 0.934 - ETA: 31s - loss: 0.1763 - accuracy: 0.934 - ETA: 31s - loss: 0.1764 - accuracy: 0.934 - ETA: 31s - loss: 0.1762 - accuracy: 0.934 - ETA: 31s - loss: 0.1758 - accuracy: 0.935 - ETA: 30s - loss: 0.1764 - accuracy: 0.934 - ETA: 30s - loss: 0.1766 - acc

23936/25000 [===========================>..] - ETA: 17s - loss: 0.1799 - accuracy: 0.932 - ETA: 17s - loss: 0.1800 - accuracy: 0.932 - ETA: 17s - loss: 0.1801 - accuracy: 0.932 - ETA: 17s - loss: 0.1800 - accuracy: 0.932 - ETA: 17s - loss: 0.1801 - accuracy: 0.932 - ETA: 17s - loss: 0.1801 - accuracy: 0.932 - ETA: 17s - loss: 0.1801 - accuracy: 0.932 - ETA: 17s - loss: 0.1801 - accuracy: 0.932 - ETA: 17s - loss: 0.1800 - accuracy: 0.932 - ETA: 17s - loss: 0.1799 - accuracy: 0.932 - ETA: 17s - loss: 0.1801 - accuracy: 0.932 - ETA: 16s - loss: 0.1799 - accuracy: 0.932 - ETA: 16s - loss: 0.1799 - accuracy: 0.932 - ETA: 16s - loss: 0.1802 - accuracy: 0.932 - ETA: 16s - loss: 0.1803 - accuracy: 0.932 - ETA: 16s - loss: 0.1801 - accuracy: 0.932 - ETA: 16s - loss: 0.1801 - accuracy: 0.932 - ETA: 16s - loss: 0.1801 - accuracy: 0.932 - ETA: 16s - loss: 0.1803 - accuracy: 0.932 - ETA: 16s - loss: 0.1804 - accuracy: 0.932 - ETA: 16s - loss: 0.1805 - accuracy: 0.932 - ETA: 16s - loss: 0.1805 - acc

25000/25000 [==============================] - ETA: 2s - loss: 0.1818 - accuracy: 0.93 - ETA: 2s - loss: 0.1819 - accuracy: 0.93 - ETA: 2s - loss: 0.1822 - accuracy: 0.93 - ETA: 2s - loss: 0.1823 - accuracy: 0.93 - ETA: 2s - loss: 0.1821 - accuracy: 0.93 - ETA: 2s - loss: 0.1821 - accuracy: 0.93 - ETA: 2s - loss: 0.1820 - accuracy: 0.93 - ETA: 2s - loss: 0.1820 - accuracy: 0.93 - ETA: 1s - loss: 0.1820 - accuracy: 0.93 - ETA: 1s - loss: 0.1819 - accuracy: 0.93 - ETA: 1s - loss: 0.1819 - accuracy: 0.93 - ETA: 1s - loss: 0.1818 - accuracy: 0.93 - ETA: 1s - loss: 0.1817 - accuracy: 0.93 - ETA: 1s - loss: 0.1816 - accuracy: 0.93 - ETA: 1s - loss: 0.1816 - accuracy: 0.93 - ETA: 1s - loss: 0.1815 - accuracy: 0.93 - ETA: 1s - loss: 0.1816 - accuracy: 0.93 - ETA: 1s - loss: 0.1818 - accuracy: 0.93 - ETA: 1s - loss: 0.1817 - accuracy: 0.93 - ETA: 1s - loss: 0.1819 - accuracy: 0.93 - ETA: 0s - loss: 0.1820 - accuracy: 0.93 - ETA: 0s - loss: 0.1819 - accuracy: 0.93 - ETA: 0s - loss: 0.1824 - accu

# evaluate the model_2

In [46]:
model_2.evaluate(X_test, y_test)

17824/25000 [====================>.........] - ETA: 5:03 - loss: 0.6329 - accuracy: 0.81 - ETA: 1:26 - loss: 0.4471 - accuracy: 0.82 - ETA: 55s - loss: 0.4622 - accuracy: 0.8348 - ETA: 43s - loss: 0.5157 - accuracy: 0.809 - ETA: 37s - loss: 0.5448 - accuracy: 0.793 - ETA: 33s - loss: 0.5658 - accuracy: 0.787 - ETA: 30s - loss: 0.5496 - accuracy: 0.791 - ETA: 28s - loss: 0.5422 - accuracy: 0.795 - ETA: 26s - loss: 0.5413 - accuracy: 0.797 - ETA: 25s - loss: 0.5259 - accuracy: 0.804 - ETA: 24s - loss: 0.5371 - accuracy: 0.797 - ETA: 23s - loss: 0.5312 - accuracy: 0.801 - ETA: 23s - loss: 0.5216 - accuracy: 0.804 - ETA: 22s - loss: 0.5163 - accuracy: 0.806 - ETA: 21s - loss: 0.5193 - accuracy: 0.806 - ETA: 21s - loss: 0.5245 - accuracy: 0.805 - ETA: 21s - loss: 0.5186 - accuracy: 0.804 - ETA: 20s - loss: 0.5066 - accuracy: 0.808 - ETA: 20s - loss: 0.5091 - accuracy: 0.808 - ETA: 20s - loss: 0.5192 - accuracy: 0.805 - ETA: 19s - loss: 0.5189 - accuracy: 0.804 - ETA: 19s - loss: 0.5181 - ac

25000/25000 [==============================] - ETA: 5s - loss: 0.5060 - accuracy: 0.80 - ETA: 5s - loss: 0.5064 - accuracy: 0.80 - ETA: 4s - loss: 0.5065 - accuracy: 0.80 - ETA: 4s - loss: 0.5064 - accuracy: 0.80 - ETA: 4s - loss: 0.5068 - accuracy: 0.80 - ETA: 4s - loss: 0.5071 - accuracy: 0.80 - ETA: 4s - loss: 0.5065 - accuracy: 0.80 - ETA: 4s - loss: 0.5059 - accuracy: 0.80 - ETA: 4s - loss: 0.5062 - accuracy: 0.80 - ETA: 4s - loss: 0.5056 - accuracy: 0.80 - ETA: 4s - loss: 0.5049 - accuracy: 0.80 - ETA: 4s - loss: 0.5052 - accuracy: 0.80 - ETA: 4s - loss: 0.5050 - accuracy: 0.80 - ETA: 4s - loss: 0.5044 - accuracy: 0.80 - ETA: 4s - loss: 0.5042 - accuracy: 0.80 - ETA: 4s - loss: 0.5046 - accuracy: 0.80 - ETA: 3s - loss: 0.5042 - accuracy: 0.80 - ETA: 3s - loss: 0.5043 - accuracy: 0.80 - ETA: 3s - loss: 0.5051 - accuracy: 0.80 - ETA: 3s - loss: 0.5052 - accuracy: 0.80 - ETA: 3s - loss: 0.5046 - accuracy: 0.80 - ETA: 3s - loss: 0.5047 - accuracy: 0.80 - ETA: 3s - loss: 0.5047 - accu

[0.5072451634597779, 0.80276]

# build the neural network model_3 - bidirectional lstm

In [48]:
embedding_dim=16

model_3 = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.Bidirectional(layers.LSTM(128)),
  layers.Dense(16, activation='relu'),
  layers.Dense(1)
])

model_3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               148480    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                4112      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 283,569
Trainable params: 283,569
Non-trainable params: 0
_________________________________________________________________


# compile the model_3

In [49]:
model_3.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# train the model_3

In [50]:
history_3 = model_3.fit(X_train, y_train, epochs=5)

Train on 25000 samples
Epoch 1/5


 5920/25000 [======>.......................] - ETA: 31:08 - loss: 0.6934 - accuracy: 0.500 - ETA: 16:06 - loss: 0.6933 - accuracy: 0.515 - ETA: 11:06 - loss: 0.6931 - accuracy: 0.552 - ETA: 8:35 - loss: 0.6932 - accuracy: 0.531 - ETA: 7:05 - loss: 0.6934 - accuracy: 0.51 - ETA: 6:05 - loss: 0.6931 - accuracy: 0.52 - ETA: 5:22 - loss: 0.6930 - accuracy: 0.53 - ETA: 4:49 - loss: 0.6927 - accuracy: 0.54 - ETA: 4:24 - loss: 0.6929 - accuracy: 0.53 - ETA: 4:04 - loss: 0.6929 - accuracy: 0.53 - ETA: 3:48 - loss: 0.6936 - accuracy: 0.51 - ETA: 3:34 - loss: 0.6937 - accuracy: 0.50 - ETA: 3:22 - loss: 0.6934 - accuracy: 0.51 - ETA: 3:12 - loss: 0.6938 - accuracy: 0.50 - ETA: 3:03 - loss: 0.6939 - accuracy: 0.49 - ETA: 2:55 - loss: 0.6940 - accuracy: 0.49 - ETA: 2:48 - loss: 0.6940 - accuracy: 0.49 - ETA: 2:42 - loss: 0.6939 - accuracy: 0.49 - ETA: 2:36 - loss: 0.6939 - accuracy: 0.49 - ETA: 2:31 - loss: 0.6940 - accuracy: 0.47 - ETA: 2:27 - loss: 0.6939 - accuracy: 0.47 - ETA: 2:22 - loss: 0.69

11872/25000 [=============>................] - ETA: 56s - loss: 0.6845 - accuracy: 0.508 - ETA: 56s - loss: 0.6841 - accuracy: 0.509 - ETA: 56s - loss: 0.6839 - accuracy: 0.509 - ETA: 56s - loss: 0.6827 - accuracy: 0.511 - ETA: 55s - loss: 0.6824 - accuracy: 0.512 - ETA: 55s - loss: 0.6821 - accuracy: 0.514 - ETA: 55s - loss: 0.6820 - accuracy: 0.515 - ETA: 55s - loss: 0.6814 - accuracy: 0.516 - ETA: 55s - loss: 0.6813 - accuracy: 0.516 - ETA: 55s - loss: 0.6812 - accuracy: 0.517 - ETA: 55s - loss: 0.6808 - accuracy: 0.519 - ETA: 54s - loss: 0.6805 - accuracy: 0.519 - ETA: 54s - loss: 0.6803 - accuracy: 0.520 - ETA: 54s - loss: 0.6806 - accuracy: 0.521 - ETA: 54s - loss: 0.6802 - accuracy: 0.521 - ETA: 54s - loss: 0.6799 - accuracy: 0.522 - ETA: 54s - loss: 0.6797 - accuracy: 0.523 - ETA: 54s - loss: 0.6793 - accuracy: 0.524 - ETA: 54s - loss: 0.6788 - accuracy: 0.525 - ETA: 53s - loss: 0.6786 - accuracy: 0.525 - ETA: 53s - loss: 0.6786 - accuracy: 0.525 - ETA: 53s - loss: 0.6784 - acc

17824/25000 [====================>.........] - ETA: 37s - loss: 0.6224 - accuracy: 0.607 - ETA: 37s - loss: 0.6220 - accuracy: 0.608 - ETA: 37s - loss: 0.6213 - accuracy: 0.608 - ETA: 37s - loss: 0.6208 - accuracy: 0.608 - ETA: 37s - loss: 0.6203 - accuracy: 0.609 - ETA: 37s - loss: 0.6197 - accuracy: 0.609 - ETA: 37s - loss: 0.6194 - accuracy: 0.610 - ETA: 37s - loss: 0.6191 - accuracy: 0.610 - ETA: 37s - loss: 0.6185 - accuracy: 0.611 - ETA: 37s - loss: 0.6182 - accuracy: 0.611 - ETA: 36s - loss: 0.6183 - accuracy: 0.611 - ETA: 36s - loss: 0.6182 - accuracy: 0.612 - ETA: 36s - loss: 0.6179 - accuracy: 0.612 - ETA: 36s - loss: 0.6176 - accuracy: 0.612 - ETA: 36s - loss: 0.6173 - accuracy: 0.613 - ETA: 36s - loss: 0.6168 - accuracy: 0.613 - ETA: 36s - loss: 0.6167 - accuracy: 0.614 - ETA: 36s - loss: 0.6163 - accuracy: 0.614 - ETA: 36s - loss: 0.6157 - accuracy: 0.615 - ETA: 36s - loss: 0.6156 - accuracy: 0.615 - ETA: 36s - loss: 0.6153 - accuracy: 0.615 - ETA: 35s - loss: 0.6146 - acc

23872/25000 [===========================>..] - ETA: 20s - loss: 0.5675 - accuracy: 0.666 - ETA: 20s - loss: 0.5673 - accuracy: 0.666 - ETA: 20s - loss: 0.5671 - accuracy: 0.666 - ETA: 20s - loss: 0.5670 - accuracy: 0.666 - ETA: 20s - loss: 0.5670 - accuracy: 0.666 - ETA: 20s - loss: 0.5669 - accuracy: 0.666 - ETA: 20s - loss: 0.5670 - accuracy: 0.666 - ETA: 20s - loss: 0.5670 - accuracy: 0.666 - ETA: 20s - loss: 0.5668 - accuracy: 0.666 - ETA: 20s - loss: 0.5666 - accuracy: 0.666 - ETA: 19s - loss: 0.5664 - accuracy: 0.666 - ETA: 19s - loss: 0.5663 - accuracy: 0.666 - ETA: 19s - loss: 0.5660 - accuracy: 0.666 - ETA: 19s - loss: 0.5657 - accuracy: 0.667 - ETA: 19s - loss: 0.5654 - accuracy: 0.667 - ETA: 19s - loss: 0.5650 - accuracy: 0.667 - ETA: 19s - loss: 0.5647 - accuracy: 0.668 - ETA: 19s - loss: 0.5642 - accuracy: 0.668 - ETA: 19s - loss: 0.5635 - accuracy: 0.669 - ETA: 19s - loss: 0.5635 - accuracy: 0.669 - ETA: 18s - loss: 0.5633 - accuracy: 0.669 - ETA: 18s - loss: 0.5631 - acc

...

 5952/25000 [======>.......................] - ETA: 1:09 - loss: 0.0603 - accuracy: 1.00 - ETA: 1:12 - loss: 0.0971 - accuracy: 0.96 - ETA: 1:13 - loss: 0.1142 - accuracy: 0.97 - ETA: 1:15 - loss: 0.1485 - accuracy: 0.96 - ETA: 1:16 - loss: 0.1605 - accuracy: 0.95 - ETA: 1:15 - loss: 0.1487 - accuracy: 0.95 - ETA: 1:15 - loss: 0.1695 - accuracy: 0.95 - ETA: 1:16 - loss: 0.1578 - accuracy: 0.95 - ETA: 1:16 - loss: 0.1540 - accuracy: 0.95 - ETA: 1:16 - loss: 0.1571 - accuracy: 0.95 - ETA: 1:15 - loss: 0.1538 - accuracy: 0.95 - ETA: 1:15 - loss: 0.1469 - accuracy: 0.96 - ETA: 1:15 - loss: 0.1479 - accuracy: 0.95 - ETA: 1:14 - loss: 0.1411 - accuracy: 0.95 - ETA: 1:15 - loss: 0.1461 - accuracy: 0.95 - ETA: 1:15 - loss: 0.1431 - accuracy: 0.96 - ETA: 1:15 - loss: 0.1439 - accuracy: 0.95 - ETA: 1:15 - loss: 0.1424 - accuracy: 0.96 - ETA: 1:15 - loss: 0.1365 - accuracy: 0.96 - ETA: 1:16 - loss: 0.1419 - accuracy: 0.95 - ETA: 1:16 - loss: 0.1462 - accuracy: 0.95 - ETA: 1:16 - loss: 0.1445 - ac

11904/25000 [=============>................] - ETA: 1:03 - loss: 0.1274 - accuracy: 0.95 - ETA: 1:03 - loss: 0.1270 - accuracy: 0.95 - ETA: 1:03 - loss: 0.1265 - accuracy: 0.95 - ETA: 1:02 - loss: 0.1261 - accuracy: 0.95 - ETA: 1:02 - loss: 0.1270 - accuracy: 0.95 - ETA: 1:02 - loss: 0.1264 - accuracy: 0.95 - ETA: 1:02 - loss: 0.1262 - accuracy: 0.95 - ETA: 1:02 - loss: 0.1267 - accuracy: 0.95 - ETA: 1:02 - loss: 0.1269 - accuracy: 0.95 - ETA: 1:02 - loss: 0.1268 - accuracy: 0.95 - ETA: 1:02 - loss: 0.1270 - accuracy: 0.95 - ETA: 1:01 - loss: 0.1273 - accuracy: 0.95 - ETA: 1:01 - loss: 0.1271 - accuracy: 0.95 - ETA: 1:01 - loss: 0.1266 - accuracy: 0.95 - ETA: 1:01 - loss: 0.1267 - accuracy: 0.95 - ETA: 1:01 - loss: 0.1263 - accuracy: 0.95 - ETA: 1:01 - loss: 0.1267 - accuracy: 0.95 - ETA: 1:01 - loss: 0.1262 - accuracy: 0.95 - ETA: 1:01 - loss: 0.1257 - accuracy: 0.95 - ETA: 1:00 - loss: 0.1256 - accuracy: 0.95 - ETA: 1:00 - loss: 0.1253 - accuracy: 0.95 - ETA: 1:00 - loss: 0.1252 - ac

17856/25000 [====================>.........] - ETA: 41s - loss: 0.1279 - accuracy: 0.953 - ETA: 41s - loss: 0.1280 - accuracy: 0.953 - ETA: 41s - loss: 0.1282 - accuracy: 0.953 - ETA: 41s - loss: 0.1282 - accuracy: 0.953 - ETA: 41s - loss: 0.1280 - accuracy: 0.953 - ETA: 41s - loss: 0.1281 - accuracy: 0.953 - ETA: 41s - loss: 0.1282 - accuracy: 0.953 - ETA: 41s - loss: 0.1284 - accuracy: 0.953 - ETA: 41s - loss: 0.1286 - accuracy: 0.953 - ETA: 41s - loss: 0.1295 - accuracy: 0.953 - ETA: 40s - loss: 0.1296 - accuracy: 0.953 - ETA: 40s - loss: 0.1293 - accuracy: 0.953 - ETA: 40s - loss: 0.1295 - accuracy: 0.953 - ETA: 40s - loss: 0.1297 - accuracy: 0.953 - ETA: 40s - loss: 0.1294 - accuracy: 0.953 - ETA: 40s - loss: 0.1295 - accuracy: 0.953 - ETA: 40s - loss: 0.1293 - accuracy: 0.953 - ETA: 40s - loss: 0.1294 - accuracy: 0.953 - ETA: 40s - loss: 0.1295 - accuracy: 0.953 - ETA: 39s - loss: 0.1294 - accuracy: 0.953 - ETA: 39s - loss: 0.1293 - accuracy: 0.953 - ETA: 39s - loss: 0.1291 - acc

23872/25000 [===========================>..] - ETA: 22s - loss: 0.1333 - accuracy: 0.952 - ETA: 22s - loss: 0.1332 - accuracy: 0.952 - ETA: 22s - loss: 0.1332 - accuracy: 0.952 - ETA: 22s - loss: 0.1334 - accuracy: 0.952 - ETA: 22s - loss: 0.1332 - accuracy: 0.952 - ETA: 22s - loss: 0.1334 - accuracy: 0.952 - ETA: 22s - loss: 0.1336 - accuracy: 0.951 - ETA: 22s - loss: 0.1336 - accuracy: 0.951 - ETA: 22s - loss: 0.1339 - accuracy: 0.951 - ETA: 22s - loss: 0.1338 - accuracy: 0.951 - ETA: 21s - loss: 0.1340 - accuracy: 0.951 - ETA: 21s - loss: 0.1341 - accuracy: 0.951 - ETA: 21s - loss: 0.1340 - accuracy: 0.951 - ETA: 21s - loss: 0.1339 - accuracy: 0.951 - ETA: 21s - loss: 0.1338 - accuracy: 0.951 - ETA: 21s - loss: 0.1337 - accuracy: 0.951 - ETA: 21s - loss: 0.1338 - accuracy: 0.951 - ETA: 21s - loss: 0.1337 - accuracy: 0.951 - ETA: 21s - loss: 0.1337 - accuracy: 0.951 - ETA: 21s - loss: 0.1337 - accuracy: 0.951 - ETA: 20s - loss: 0.1335 - accuracy: 0.951 - ETA: 20s - loss: 0.1334 - acc

25000/25000 [==============================] - ETA: 3s - loss: 0.1374 - accuracy: 0.95 - ETA: 3s - loss: 0.1374 - accuracy: 0.95 - ETA: 3s - loss: 0.1373 - accuracy: 0.95 - ETA: 3s - loss: 0.1373 - accuracy: 0.95 - ETA: 3s - loss: 0.1375 - accuracy: 0.95 - ETA: 3s - loss: 0.1374 - accuracy: 0.95 - ETA: 2s - loss: 0.1375 - accuracy: 0.95 - ETA: 2s - loss: 0.1375 - accuracy: 0.95 - ETA: 2s - loss: 0.1376 - accuracy: 0.95 - ETA: 2s - loss: 0.1377 - accuracy: 0.95 - ETA: 2s - loss: 0.1379 - accuracy: 0.95 - ETA: 2s - loss: 0.1379 - accuracy: 0.95 - ETA: 2s - loss: 0.1379 - accuracy: 0.95 - ETA: 2s - loss: 0.1380 - accuracy: 0.95 - ETA: 2s - loss: 0.1381 - accuracy: 0.95 - ETA: 1s - loss: 0.1381 - accuracy: 0.95 - ETA: 1s - loss: 0.1381 - accuracy: 0.95 - ETA: 1s - loss: 0.1383 - accuracy: 0.95 - ETA: 1s - loss: 0.1383 - accuracy: 0.95 - ETA: 1s - loss: 0.1386 - accuracy: 0.95 - ETA: 1s - loss: 0.1386 - accuracy: 0.95 - ETA: 1s - loss: 0.1387 - accuracy: 0.95 - ETA: 1s - loss: 0.1385 - accu

# validate the model_3

In [51]:
model_3.evaluate(X_test, y_test)

18176/25000 [====================>.........] - ETA: 6:59 - loss: 0.7449 - accuracy: 0.81 - ETA: 1:55 - loss: 0.5655 - accuracy: 0.81 - ETA: 1:11 - loss: 0.5391 - accuracy: 0.81 - ETA: 54s - loss: 0.6433 - accuracy: 0.7750 - ETA: 44s - loss: 0.6332 - accuracy: 0.774 - ETA: 39s - loss: 0.6640 - accuracy: 0.763 - ETA: 35s - loss: 0.6584 - accuracy: 0.769 - ETA: 32s - loss: 0.6377 - accuracy: 0.777 - ETA: 29s - loss: 0.6228 - accuracy: 0.781 - ETA: 28s - loss: 0.6020 - accuracy: 0.792 - ETA: 26s - loss: 0.6176 - accuracy: 0.787 - ETA: 25s - loss: 0.6080 - accuracy: 0.790 - ETA: 24s - loss: 0.5992 - accuracy: 0.792 - ETA: 23s - loss: 0.5981 - accuracy: 0.792 - ETA: 22s - loss: 0.6030 - accuracy: 0.788 - ETA: 22s - loss: 0.6085 - accuracy: 0.786 - ETA: 21s - loss: 0.6026 - accuracy: 0.788 - ETA: 21s - loss: 0.5917 - accuracy: 0.793 - ETA: 20s - loss: 0.5973 - accuracy: 0.793 - ETA: 20s - loss: 0.6090 - accuracy: 0.789 - ETA: 19s - loss: 0.6024 - accuracy: 0.790 - ETA: 19s - loss: 0.5994 - ac

25000/25000 [==============================] - ETA: 4s - loss: 0.5784 - accuracy: 0.80 - ETA: 4s - loss: 0.5783 - accuracy: 0.80 - ETA: 4s - loss: 0.5781 - accuracy: 0.80 - ETA: 3s - loss: 0.5779 - accuracy: 0.80 - ETA: 3s - loss: 0.5782 - accuracy: 0.80 - ETA: 3s - loss: 0.5786 - accuracy: 0.80 - ETA: 3s - loss: 0.5784 - accuracy: 0.80 - ETA: 3s - loss: 0.5791 - accuracy: 0.80 - ETA: 3s - loss: 0.5781 - accuracy: 0.80 - ETA: 3s - loss: 0.5774 - accuracy: 0.80 - ETA: 3s - loss: 0.5767 - accuracy: 0.80 - ETA: 3s - loss: 0.5779 - accuracy: 0.80 - ETA: 3s - loss: 0.5781 - accuracy: 0.80 - ETA: 3s - loss: 0.5778 - accuracy: 0.80 - ETA: 3s - loss: 0.5781 - accuracy: 0.80 - ETA: 3s - loss: 0.5783 - accuracy: 0.80 - ETA: 3s - loss: 0.5775 - accuracy: 0.80 - ETA: 3s - loss: 0.5772 - accuracy: 0.80 - ETA: 3s - loss: 0.5771 - accuracy: 0.80 - ETA: 3s - loss: 0.5781 - accuracy: 0.80 - ETA: 2s - loss: 0.5775 - accuracy: 0.80 - ETA: 2s - loss: 0.5781 - accuracy: 0.80 - ETA: 2s - loss: 0.5778 - accu

[0.5788243908023835, 0.80096]